In [ ]:
import time

while True:
    print("Ejecutando instrucción")
    time.sleep(5)  # Pausa de 100 segundos

Ejecutando instrucción
Ejecutando instrucción
Ejecutando instrucción
Ejecutando instrucción
Ejecutando instrucción
Ejecutando instrucción
Ejecutando instrucción
Ejecutando instrucción


In [ ]:
from heyoo import WhatsApp

In [1]:
def enviar():
    #TOKEN DE ACCESO DE FACEBOOK
    token='EAAUZA7GQszZAABAG5q4EZCHue0kHyxatPoIM5VzrDx0TxGODBdKrP7eTXDqjJsSGx9pTdnU9p9dxI9eUwYxwqCFTcZAvwvvzsxC5BzYIYwKFfgOZBxfAicxifcYbX1KYBokYiOgIzCdv54rybBRuc5XhiMOWgOGgK6ZCPgzCs87MIYvpquNlW4gjGafbYHIPjr3mKZCb9PWkVpxfoa3CJZCd1iIZAIjy8K7YZD'
    #IDENTIFICADOR DE NÚMERO DE TELÉFONO
    idNumeroTeléfono='110769695347161'
    #TELEFONO QUE RECIBE (EL DE NOSOTROS QUE DIMOS DE ALTA)
    telefonoEnvia='59178194371'
    #MENSAJE A ENVIAR
    textoMensaje="Hola novato saludos"
    #URL DE LA IMAGEN A ENVIAR
    #urlImagen='https://i.imgur.com/r5lhxgn.png'
    #INICIALIZAMOS ENVIO DE MENSAJES
    mensajeWa=WhatsApp(token,idNumeroTeléfono)
    #ENVIAMOS UN MENSAJE DE TEXTO
    mensajeWa.send_message(textoMensaje,telefonoEnvia)
    #ENVIAMOS UNA IMAGEN
    #mensajeWa.send_image(image=urlImagen,recipient_id=telefonoEnvia,)
    return "mensaje enviado exitosamente"